In [15]:
# if you use word embeddings to represent text, provide the script for loading the embeddings below
# if you use glove, word2vec or fasttext embeddings, please specify which version you use (e.g. glove.6B.300d)
# if you use other embedding models, please provide the download link

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

path_of_downloaded_files = "/MLOps/2.Propaganda_detection/glove.6B.300d.txt"
glove_file = datapath(path_of_downloaded_files)
word2vec_glove_file = get_tmpfile("glove.6B.300d.txt")
glove2word2vec(glove_file, word2vec_glove_file)
word_vectors = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [16]:
# scripts for creating sentence vectors; adjust the code if necessary

from nltk.tokenize import word_tokenize
def vectorize_sent(word_vectors, sent, oov_vec):
    word_vecs = []
    for token in word_tokenize(sent): 
        if token not in word_vectors: 
            word_vecs.append(oov_vec)
        else:
            word_vecs.append(word_vectors[token].astype('float64'))
    return np.mean(word_vecs,axis=0)

In [14]:
# NOTE! The model defined below MUST BE EXACTLY THE SAME as the one you used at training
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim, out_dim, dp_rate):
        super(MLP, self).__init__()
        self.hidden_layer = nn.Linear(input_dim, input_dim*2)
        self.output_layer = nn.Linear(input_dim*2, out_dim)
        self.dropout = nn.Dropout(dp_rate)
        self.relu = torch.nn.ReLU()

    def forward(self, x_in):
        z1 = self.dropout(x_in) # output of the input layer, after dropout
        z2 = self.relu(self.hidden_layer(z1)) # output of the hidden layer
        logits = self.output_layer(z2)
        return logits

In [17]:
# reconstruct your trained model from pickle

import pickle
def reconstruct_model(pickle_path):
    saved_model_dic = pickle.load(open(pickle_path,"rb"))
    input_dim = saved_model_dic['input_dim']
    dp_rate = saved_model_dic['dropout_rate']
    output_dim = 2
    model = MLP(input_dim, output_dim, dp_rate)
    saved_weights = saved_model_dic['neural_weights']
    model.load_state_dict(saved_weights)
    oov_vec = saved_model_dic['oov_vector']
    
    return model, oov_vec

In [18]:
# use the reconstructed model to make predictions on the test data

def test_trained_model(model, oov_vec, test_text):
    test_vecs = [vectorize_sent(word_vectors, ss, oov_vec ) for ss in test_text]
    test_vecs_tensor = torch.tensor(test_vecs, dtype=torch.float)
    test_prediction = model(test_vecs_tensor)
    pred_labels = [np.argmax(tp.detach().numpy()) for tp in test_prediction]
    return pred_labels

In [19]:
# load sample test data
import pandas as pd
import numpy as np
test_data = pd.read_table('train.tsv')
test_text = test_data['sentence_text'].tolist()[-2000:]
test_raw_labels = test_data['label'].tolist()[-2000:]
label_dic = {'non-propaganda':0, 'propaganda':1}
test_labels = [label_dic[rl] for rl in test_raw_labels]

print('test data size', len(test_labels))

# reconstruct model and make predictions
model, oov_vec = reconstruct_model('propoganda.pickle')
test_pred = test_trained_model(model, oov_vec, test_text)

# test model
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
pre, rec, f1, _ = precision_recall_fscore_support(test_labels, test_pred, average='macro')
print('macro-F1 on test data', f1)

test data size 2000
macro-F1 on test data 0.6169517780172413


In [21]:
def predict_query(model, word_vectors, oov_vec, query):
    query_vec = vectorize_sent(word_vectors, query, oov_vec)
    query_tensor = torch.tensor([query_vec], dtype=torch.float)  # Add batch dimension
    with torch.no_grad():
        model.eval()
        prediction = model(query_tensor)
        pred_label = torch.argmax(prediction, dim=1).item()
    return pred_label

In [28]:
model, oov_vec = reconstruct_model('propoganda.pickle')

if __name__ == "__main__":
    query = input("Enter the query: ")
    pred_labels = predict_query(model,word_vectors, oov_vec, query)
    label_map = {0: "non-propaganda", 1: "propaganda"}
    print(f"{label_map[pred_labels]}")


non-propaganda
